<h1>Pacakages</h1>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import networkx as nx
from tqdm.notebook import tqdm
tqdm.pandas()
import os

<h1>Auxiliary functions to clean the bot</h1>

In [2]:
bot_list = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/bot_list.csv')
bot_list['Username'] = bot_list.Username.apply(lambda x: x[3:])

In [3]:
popular_author = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/10k_sub 2.csv')

popular_bot = popular_author[popular_author.is_bot == 1]
popular_bot.head()

,author,num_comments,num_del_comments,num_posts,num_del_posts,bot_flag,all_sub,is_bot,Unnamed: 8
0,AutoModerator,289569.0,0.0,670.0,7.0,0,290239.0,1,NaN
1,AveenoFresh,11756.0,0.0,25.0,2.0,0,11781.0,1,NaN
3,FannyPackPhantom,13549.0,0.0,0.0,0.0,0,13549.0,1,NaN
5,G23456789,11965.0,0.0,2.0,2.0,0,11967.0,1,NaN
6,GoBeaversOSU,26183.0,2.0,6.0,0.0,0,26189.0,1,NaN


In [4]:
def advanced_clean(name):
    
    tokens = re.findall(r'[A-Z](?:[A-Z]*(?![a-z])|[a-z]*)', name)
    tokens = list(map(lambda x: x.lower(), tokens))
    
    tokens = list (map(lambda x: re.split('[^a-zA-Z]',x), tokens))
    tokens = [item for sublist in tokens for item in sublist]
    
    if 'not' in tokens and 'bot' in tokens:
        return False

    elif 'bot' in tokens:
        return True
    
    return False

In [5]:
def full_clean (dataframe):
    # shallow clean
    dataframe = dataframe[dataframe.author != '[deleted]']
    dataframe = dataframe[dataframe.author != 'AutoModerator']
    dataframe = dataframe[dataframe.author != 'VisualMod']
    
    # deep clean 
    dataframe['is_bot']  = dataframe['author'].progress_apply(advanced_clean)
    bot_df1 = dataframe[dataframe.is_bot == True]
    dataframe = dataframe[dataframe.is_bot == False]
    
    # further clean
    dataframe['is_bot'] = dataframe.author.isin(bot_list.Username)
    bot_df2 = dataframe[dataframe.is_bot == True]
    dataframe = dataframe[dataframe.is_bot == False]
    
    dataframe['is_bot'] = dataframe.author.isin(popular_bot.author)
    bot_df3 = dataframe[dataframe.is_bot == True]
    dataframe = dataframe[dataframe.is_bot == False]

    
    bot = pd.concat([bot_df1, bot_df2,bot_df3])
    bot = bot.drop_duplicates()
    
    return bot, dataframe
    
    

<h1>Auxilary functions</h1>

In [6]:
def create_name_id_dict(dataframe):    #create id:author dataframe
    name_id = {}
    for i in range(dataframe.shape[0]):
        key = dataframe.id.iloc[i]
        value = dataframe.author.iloc[i]

        key_value_pair = {key:value}

        name_id.update(key_value_pair)
    return name_id

<h1>Read the 1.5 yrs data</h1>

In [ ]:
fp = '/Users/victor/Desktop/CS/UST_summer/extracted_cols.csv'
selected_cols = ['author', 'id', 'link_id', 'parent_id']
extracted_cols = pd.read_csv(fp, low_memory = False)[selected_cols]

bot_extracted_cols, extracted_cols = full_clean(extracted_cols)

In [ ]:
# save extracted_cols bot
bot_extracted_cols.to_csv('bot_extracted_cols.csv')

<h1>For t3-df</h1>

In [7]:
fp = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv' # the directory of the raw data file, pipeliend from reddits
sc = ['id', 'author']
t3_df = pd.read_csv(fp, low_memory=False, encoding='utf-8')[sc]

t3_bot, t3_df = full_clean(t3_df)

  0%|          | 0/1087302 [00:00<?, ?it/s]

In [8]:
# save bot df
t3_bot.to_csv('t3_bot.csv')

In [22]:
t3_df.head()

,id,author,is_bot
0,mhj2yd,USWD_Bank,False
2,mhj3ew,KingMacias1,False
3,mhj3oa,jt989898,False
4,mhj3vq,KenAdamsonPrime,False
6,mhj4o5,lexmark0331,False


In [9]:
#create the output dataframe for level_1 commentors
def create_level_1(dataframe):     #create the output dataframe
    level_1_comments = dataframe[dataframe['parent_id'] == dataframe['link_id']]
    t_3_posts_authors  = level_1_comments.groupby('parent_id')['author'].apply(list).reset_index(name='author')
    t_3_posts_authors['sub_comments'] = t_3_posts_authors['author'].progress_apply(lambda x:len(x))
    return t_3_posts_authors

In [10]:
fp = '/Users/victor/Desktop/CS/UST_summer/extracted_cols.csv'
selected_cols = ['author', 'id', 'link_id', 'parent_id']
extracted_cols = pd.read_csv(fp, low_memory = False)[selected_cols]

bot_extracted_cols, extracted_cols = full_clean(extracted_cols)

  0%|          | 0/32874876 [00:00<?, ?it/s]

In [11]:
# save extracted_cols bot
bot_extracted_cols.to_csv('bot_extracted_cols.csv')

In [12]:
t3_posts_authors = create_level_1(extracted_cols)
t3_posts_authors.head()

  0%|          | 0/356610 [00:00<?, ?it/s]

,parent_id,author,sub_comments
0,t3_cb7x6u,"[K3rn3l_pAn1k, Lucarioman7, 2broke2flex, DumbR...",8
1,t3_cca4bh,[ParagonHereticForce],1
2,t3_ce7f4e,[Captainflippypants],1
3,t3_chcx3a,"[The_King32, Xalteox]",2
4,t3_cj3l52,[qwaszx937],1


<h1>For t1-df</h1>

In [14]:
def create_t1_df(dataframe):
    """
    A function that creates an output dataframe for us to create the edge list later
    Args:
        daydf (pandas.dataframe): a DAY dataframe to be examined

    Returns:
        main_authors: a list of authors the gives the main post
        output_df: the output dataframe we want
    """
    level_1_comments = dataframe[dataframe['parent_id'] == dataframe['link_id']]

    sub_comments = dataframe[~dataframe.index.isin(level_1_comments.index)] #setting the sub comments being those with index that are not in level_1_comments

    t1_df  = sub_comments.groupby('parent_id')['author'].apply(list).reset_index(name='author')
    t1_df['sub_comments'] = t1_df['author'].progress_apply(lambda x:len(x))
    t1_df.sort_values('sub_comments', inplace=True, ascending=False)

    return t1_df

In [15]:
t1_df = create_t1_df(extracted_cols)
t1_df.head()

  0%|          | 0/12038149 [00:00<?, ?it/s]

,parent_id,author,sub_comments
11502549,t1_h0csj8y,"[Optionsfuckup, jackthemackattack, WeHaveBeenF...",2286
7395789,t1_gl35xpy,"[chayochan, thats_not_funny_guys, Appearance-R...",2210
9119830,t1_golywrq,"[Obliterateee, Grizzlytash82, MrProMuch, Squea...",1303
908975,t1_fkifvew,"[noemojica88, emenpy, mycochang, datastrm, gar...",1203
6947630,t1_gkqb2tu,"[SilentSakura, onlybreakingbad, Content-Necess...",1062


<h1>Form full dataframe</h1>

In [19]:
full_df = pd.concat([t1_df, t3_posts_authors])
full_df = full_df.reset_index(drop=True)
full_df

,parent_id,author,sub_comments
0,t1_h0csj8y,"[Optionsfuckup, jackthemackattack, WeHaveBeenF...",2286
1,t1_gl35xpy,"[chayochan, thats_not_funny_guys, Appearance-R...",2210
2,t1_golywrq,"[Obliterateee, Grizzlytash82, MrProMuch, Squea...",1303
3,t1_fkifvew,"[noemojica88, emenpy, mycochang, datastrm, gar...",1203
4,t1_gkqb2tu,"[SilentSakura, onlybreakingbad, Content-Necess...",1062
...,...,...,...
12394754,t3_ob8yl4,"[Softspokenclark, mrbullishinvestor]",2
12394755,t3_ob92zw,"[MarketingDifferent61, allintraders, Tradition...",3
12394756,t3_ob9eab,[Julez_Jay],1
12394757,t3_ob9kcs,"[escaflowne_spawn, PerennialTime]",2


<h1>Form the COMPLETE dataframe</h1>

In [20]:
def is_main(dataframe, t1_name_id, t3_name_id):
    main_list = []
    parent_author_list = []
    dataframe = dataframe.reset_index(drop = True)
    
    for index, row in dataframe.iterrows():
        uid = row['parent_id'].split('_')[-1]
        
        pre = row['parent_id'].split('_')[0]
        if pre == 't1':
            main_list.append(0)
            try:
                parent_author = t1_name_id[uid]
                parent_author_list.append(parent_author)
            except:
                parent_author_list.append(np.nan)
        else:
            main_list.append(1)
            try:
                parent_author = t3_name_id[uid]
                parent_author_list.append(parent_author)
            except:
                parent_author_list.append(np.nan)        

        if index == 100000: print(index)
    dataframe['main_post'] =  main_list
    dataframe['parent_author'] = parent_author_list
    return dataframe

In [23]:
complete_df = is_main(full_df, create_name_id_dict(extracted_cols), create_name_id_dict(t3_df))

100000


In [24]:
complete_df.to_csv('complete_df.csv')

<h1>For Confusion matrix</h1>

In [25]:
def confusion_matrix(dataframe):    
    no_idea_comment = []
    main_post_comment = []
    comment_comment = []
    
    for index, row in dataframe.iterrows():
        if row['main_post'] == np.nan:
            no_idea_comment += row['author']
        elif row['main_post'] == 0:
            comment_comment += row['author']
        else:
            main_post_comment += row['author']
    no_idea_comment = list(dict.fromkeys(no_idea_comment))
    main_post_comment = list(dict.fromkeys(main_post_comment))
    comment_comment = list(dict.fromkeys(comment_comment))
    return no_idea_comment, main_post_comment, comment_comment

In [30]:
Post = complete_df['parent_author'].to_numpy()

Post = list(dict.fromkeys(Post))
Post.remove(np.nan)

In [27]:
# confusion matrix
no_idea_comment, main_post_comment, comment_comment = confusion_matrix(complete_df)

In [28]:
def get_the_table(Post, no_idea_comment, main_post_comment, comment_comment):
        
    Post = set(Post)
    no_idea_comment = set(no_idea_comment)
    main_post_comment = set(main_post_comment)
    comment_comment = set(comment_comment)
    inter_1 = set.intersection(Post, no_idea_comment)
    inter_2 = set.intersection(Post, main_post_comment)
    inter_3 = set.intersection(Post, comment_comment)
    
    Post_Comment = {}
    Post_Comment = set().union(inter_1, inter_2, inter_3)
    
    Post_Only = Post - Post_Comment
    no_idea_comment_only = no_idea_comment - inter_1
    main_post_comment_only = main_post_comment - inter_2
    comment_comment_only = comment_comment - inter_3
    
    classes = ['Members who only post', 'Members who both post and reply to others', 'Members who only reply to main posts', 'Members who only reply to other comments', 'Unidentifiable commenters']
    Counts = [len(Post_Only), len(Post_Comment), len(main_post_comment_only), len(comment_comment_only), len(no_idea_comment_only)]
    type_table = pd.DataFrame(list(zip(classes, Counts)), columns = ['Types', 'Counts'])
    
    return type_table, list(Post_Only), list(Post_Comment), list(main_post_comment_only), list(comment_comment_only), list(no_idea_comment_only)

In [31]:
type_table, Post_Only, Post_Comment, main_post_comment_only, comment_comment_only, no_idea_comment_only = get_the_table(Post, no_idea_comment, main_post_comment, comment_comment)


In [32]:
type_table.to_csv('type_table.csv')

In [33]:
def types_df(p_o, p_c, m_p_c_o, c_c_o, n_i_c_o):
    authors = p_o + p_c + m_p_c_o + c_c_o + n_i_c_o
    post_only = list(np.ones(len(p_o))) + list(np.zeros(len(authors)-len(p_o)))
    post_comment = list(np.zeros(len(p_o))) + list(np.ones(len(p_c))) + list(np.zeros(len(authors)-len(p_o+p_c)))
    reply_main = list(np.zeros(len(p_o+p_c))) + list(np.ones(len(m_p_c_o))) + list(np.zeros(len(c_c_o+n_i_c_o)))
    reply_comment = list(np.zeros(len(authors)-len(c_c_o+n_i_c_o))) +list(np.ones(len(c_c_o))) +list(np.zeros(len(n_i_c_o)))
    unknown = list(np.zeros(len(authors)-len(n_i_c_o))) +list(np.ones(len(n_i_c_o)))
    
    type_df = pd.DataFrame(list(zip(authors, post_only, post_comment, reply_main, reply_comment, unknown)), columns = ['Author Name', 'Members who only post', 'Members who both post and reply to others', 'Members who only reply to main posts', 'Members who only reply to other comments', 'Unidentifiable commenters'])
    return type_df

In [34]:
type_df = types_df(Post_Only, Post_Comment, main_post_comment_only, comment_comment_only, no_idea_comment_only)

In [35]:
type_df.head()

,Author Name,Members who only post,Members who both post and reply to others,Members who only reply to main posts,Members who only reply to other comments,Unidentifiable commenters
0,immmj,1.0,0.0,0.0,0.0,0.0
1,Carehzy,1.0,0.0,0.0,0.0,0.0
2,FederalExternal2513,1.0,0.0,0.0,0.0,0.0
3,LeagueEmbarrassed194,1.0,0.0,0.0,0.0,0.0
4,Schaeffer410,1.0,0.0,0.0,0.0,0.0


<h1>Concat the bot</h1>

In [36]:
t3_bot = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/t3_bot.csv')
extracted_cols_bot = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/bot_extracted_cols.csv')

all_bot = pd.concat([t3_bot, extracted_cols_bot])
all_bot = all_bot.drop_duplicates()
all_bot.head()

,Unnamed: 0,id,author,is_bot,link_id,parent_id
0,854,mhv3i6,Tea_Bot278,True,NaN,NaN
1,3546,mioer6,Your_wifes_bfNOTaBOT,True,NaN,NaN
2,17089,mpeebo,Con_Bot_,True,NaN,NaN
3,31579,muma0v,ShotBot,True,NaN,NaN
4,45598,n0mhc6,BotDamian,True,NaN,NaN


In [37]:
all_bot.to_csv('all_bot.csv')

In [ ]:
def count(dataframe):
    count_df = dataframe.groupby(dataframe.author.tolist()).size().reset_index().rename(columns={0:'count'})
    
    count_df = count_df.drop_duplicates()
    
    return count_df['author', 'count']